# Fraud Detection – Data Cleaning & Preprocessing Pipeline

This notebook performs **data cleaning, integration, feature engineering, and preprocessing**
for fraud detection datasets:

1. **E-commerce Fraud Dataset**
2. **Credit Card Transaction Dataset**

The output of this notebook is **clean, scaled, and model-ready datasets**
saved to `data/processed`.


## Import Required Libraries

This section imports libraries for:
- Data manipulation and numerical computation
- Visualization (optional for analysis)
- Logging and reproducibility
- Feature preprocessing and scaling
- Handling imbalanced datasets


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from datetime import datetime


## Logging Configuration

Logging is used to:
- Track preprocessing steps
- Record important transformations
- Improve debugging and reproducibility


In [16]:
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


## Directory Setup

Defines paths for:
- Raw input datasets
- Processed output datasets

The processed directory is created automatically if it does not exist.


In [17]:
BASE_DIR = Path.cwd().parent
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


## Data Loading Function

This function:
- Loads CSV files from the raw data directory
- Validates file existence
- Logs the loading process


In [18]:
def load_data(filename):
    path = RAW_DIR / filename
    if not path.exists():
        logger.error(f"File not found: {path}")
        raise FileNotFoundError(f"File not found: {path}")
    logger.info(f"Loading {filename}...")
    return pd.read_csv(path)


## Data Cleaning

This step performs:
- Duplicate removal
- Missing value handling
- Timestamp conversion

Imputation strategy:
- Numerical columns → median
- Categorical columns → mode


In [19]:
def clean_data(df, dataset_name='dataset'):
    logger.info(f"Cleaning {dataset_name}...")
    initial_shape = df.shape
    
    df = df.drop_duplicates()
    if df.shape[0] < initial_shape[0]:
        logger.info(f"Dropped {initial_shape[0] - df.shape[0]} duplicate rows.")

    missing_cols = df.columns[df.isnull().any()].tolist()
    if missing_cols:
        logger.info(f"Columns with missing values: {missing_cols}")
        for col in missing_cols:
            if df[col].dtype in [np.float64, np.int64]:
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna(df[col].mode()[0])
        logger.info("Imputed missing values.")

    time_cols = ['signup_time', 'purchase_time']
    for col in time_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
            logger.info(f"Converted {col} to datetime.")
            
    return df


## Geolocation Integration

This step merges fraud transactions with country information using:
- IP address ranges
- `merge_asof` for efficient range matching

Unmatched IPs are labeled as **Unknown**.


In [20]:
def integration_geolocation(fraud_df, ip_df):
    logger.info("Integrating Geolocation data...")
    
    fraud_df['ip_address'] = fraud_df['ip_address'].astype(float).astype(int)
    ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(float).astype(int)
    ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(float).astype(int)
    
    fraud_df = fraud_df.sort_values(by='ip_address')
    ip_df = ip_df.sort_values(by='lower_bound_ip_address')
    
    merged = pd.merge_asof(
        fraud_df,
        ip_df,
        left_on='ip_address',
        right_on='lower_bound_ip_address',
        direction='backward'
    )
    
    mask = merged['ip_address'] <= merged['upper_bound_ip_address']
    merged.loc[~mask, 'country'] = 'Unknown'
    
    merged = merged.drop(columns=['lower_bound_ip_address', 'upper_bound_ip_address'])
    return merged


## Feature Engineering

New features are created to capture:
- Temporal patterns
- User behavior signals
- Fraud-related activity timing


In [21]:
def feature_engineering(df):
    logger.info("Engineering features...")
    
    if 'purchase_time' in df.columns:
        df['hour_of_day'] = df['purchase_time'].dt.hour
        df['day_of_week'] = df['purchase_time'].dt.dayofweek
    
    if 'signup_time' in df.columns and 'purchase_time' in df.columns:
        df['time_since_signup'] = (
            df['purchase_time'] - df['signup_time']
        ).dt.total_seconds()
        
    if 'user_id' in df.columns:
        df['transaction_count'] = df.groupby('user_id')['user_id'].transform('count')
        
    return df


## Feature Transformation & Scaling

This step:
- One-Hot Encodes categorical variables
- Applies Standard Scaling to numerical features

This prepares the dataset for machine learning models.


In [22]:
def transform_and_scale(df, numeric_cols, cat_cols, target_col=None):
    logger.info("Transforming and Scaling data...")
    
    if cat_cols:
        df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
        
    if numeric_cols:
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
        
    return df


## Class Imbalance Analysis

Fraud datasets are typically imbalanced.
This function logs:
- Absolute class counts
- Percentage distribution


In [23]:
def analyze_class_imbalance(y, name):
    counts = y.value_counts()
    percentage = y.value_counts(normalize=True) * 100
    logger.info(
        f"Class Imbalance for {name}:\n"
        f"{pd.concat([counts, percentage], axis=1, keys=['Count', 'Percentage'])}"
    )


## Fraud Dataset Processing Pipeline

This pipeline performs:
- Data loading and cleaning
- Geolocation integration
- Feature engineering
- Encoding and scaling
- Processed data export


In [24]:
def process_fraud_data():
    fraud = load_data('Fraud_Data.csv')
    ip_data = load_data('IpAddress_to_Country.csv')
    
    fraud = clean_data(fraud, 'Fraud_Data')
    fraud = integration_geolocation(fraud, ip_data)
    fraud = feature_engineering(fraud)
    
    analyze_class_imbalance(fraud['class'], 'Fraud_Data')
    
    cat_cols = ['source', 'browser', 'sex', 'country']
    num_cols = ['purchase_value', 'age', 'time_since_signup', 'hour_of_day', 'day_of_week']
    
    cat_cols = [c for c in cat_cols if c in fraud.columns]
    num_cols = [c for c in num_cols if c in fraud.columns]
    
    fraud_processed = transform_and_scale(fraud, num_cols, cat_cols)
    
    fraud_processed.to_csv(PROCESSED_DIR / 'Fraud_Data_Processed.csv', index=False)
    logger.info("Saved Fraud_Data_Processed.csv")
    
    return fraud_processed


## Credit Card Dataset Processing

This dataset is already anonymized.
Only scaling is applied to:
- Transaction amount
- Transaction time


In [25]:
def process_creditcard_data():
    credit = load_data('creditcard.csv')
    credit = clean_data(credit, 'creditcard')
    
    analyze_class_imbalance(credit['Class'], 'creditcard')
    
    scaler = StandardScaler()
    credit['Amount_Scaled'] = scaler.fit_transform(credit[['Amount']])
    credit['Time_Scaled'] = scaler.fit_transform(credit[['Time']])
    
    credit = credit.drop(['Amount', 'Time'], axis=1)
    
    credit.to_csv(PROCESSED_DIR / 'creditcard_Processed.csv', index=False)
    logger.info("Saved creditcard_Processed.csv")
    
    return credit


## Execute Full Preprocessing Pipeline

This cell runs preprocessing for both datasets
and generates model-ready CSV files.


In [26]:
process_fraud_data()
process_creditcard_data()

2025-12-23 14:45:08,380 - INFO - Loading Fraud_Data.csv...
2025-12-23 14:45:08,628 - INFO - Loading IpAddress_to_Country.csv...
2025-12-23 14:45:08,673 - INFO - Cleaning Fraud_Data...
2025-12-23 14:45:08,861 - INFO - Converted signup_time to datetime.
2025-12-23 14:45:08,904 - INFO - Converted purchase_time to datetime.
2025-12-23 14:45:08,909 - INFO - Integrating Geolocation data...
2025-12-23 14:45:08,977 - INFO - Engineering features...
2025-12-23 14:45:09,016 - INFO - Class Imbalance for Fraud_Data:
        Count  Percentage
class                    
0      136961   90.635423
1       14151    9.364577
2025-12-23 14:45:09,016 - INFO - Transforming and Scaling data...
2025-12-23 14:45:14,077 - INFO - Saved Fraud_Data_Processed.csv
2025-12-23 14:45:14,087 - INFO - Loading creditcard.csv...
2025-12-23 14:45:15,078 - INFO - Cleaning creditcard...
2025-12-23 14:45:15,779 - INFO - Dropped 1081 duplicate rows.
2025-12-23 14:45:15,795 - INFO - Class Imbalance for creditcard:
        Count  

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,Amount_Scaled,Time_Scaled
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244200,-1.996823
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342584,-1.996823
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.158900,-1.996802
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.139886,-1.996802
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073813,-1.996781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0,-0.350252,1.642235
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0,-0.254325,1.642257
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0,-0.082239,1.642278
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,0,-0.313391,1.642278
